In [14]:
%load_ext autoreload
%autoreload 2

from sentence_transformers import SentenceTransformer
from src.text_embedder.embedder import text_to_embedding_transformer

import pandas as pd
from loguru import logger

from src.text_embedder.embedder import EmbeddingsModel

from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [15]:
notebook_path = Path().resolve().parent
print(notebook_path)

# Construct the filename relative to the new path
filename = notebook_path / "test_frames_gt_new.csv"

# Read the CSV file
df = pd.read_csv(filename)
print(df)

C:\Users\baatout\PycharmProjects\afm-vlm\experiments\second_experiment
    ID                                             Prompt Length  GT_Keyframe
0    1                                  learning outcomes  short            3
1    2   What are the learning outcomes from the lecture?   long            3
2    3       What is the relation of protien and muscles?  short           26
3    4                              Essential fatty acids  short           29
4    5                                  fish oil capsules  short           30
5    6  what is the functioning of fish oil capsules a...   long           30
6    7                           calcium as micronutrient  short           46
7    8       Explain where vitamins function as coenzymes   long           41
8    9  Give a list of food items where I can attain s...   long           45
9   10          what is calcium commonly supplemented for   long           46
10  11       How are nutrients extracted from plant cells   long       

In [16]:
# create dataframe to save results of the experiments
columns = ['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3']

df_test = pd.DataFrame(columns=columns)

df_ocr_only = df_test
df_ocr_lava = df_test
df_ocr_transcriptions = df_test

df_short_llm_summary = df_test
df_extensive_summary = df_test

In [17]:
def add_to_df(prompt, gt_keyframe, result):
    # Create a new row with the provided data
    return {
        'Prompt': prompt,
        'GT_Keyframe': gt_keyframe,
        'Top_1': extract_keyframe_number(result[0]) if len(result) > 0 else None,
        'Top_2': extract_keyframe_number(result[1]) if len(result) > 1 else None,
        'Top_3': extract_keyframe_number(result[2]) if len(result) > 2 else None
    }


# ietrate over the dataframe and get the results
def get_results(df):
    for _, row in df.iterrows():
        logger.info(row['Prompt'])
        prompt = row['Prompt']
        gt_keyframe = row['GT_Keyframe']

        # Search for similar images
        output = embedding_model.search_similar_images_top_k(prompt, gt_keyframe, 3)
        res_row = add_to_df(prompt, gt_keyframe, output)
        rows.append(res_row)

    return rows


def extract_keyframe_number(path):
    """
    Extracts the scene number from the given file path.

    Parameters:
    path (str): The full path of the file.

    Returns:
    str: The extracted scene number.
    """
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]

    # Extract '032' from filename
    scene_number = filename.split('-Scene-')[-1].split('-')[0]

    return scene_number

In [146]:
#top_3 = []
#embedding_model.check_proximity_keyframes(90, top_3)

# LOAD MODEL 

In [18]:
embedding_model = EmbeddingsModel()

<Figure size 800x2000 with 0 Axes>

In [19]:
csv = notebook_path / "biology_chapter_3_3_treshhold_5.csv"
df_data_extensive = pd.read_csv(csv)
df_data_extensive

,Unnamed: 0,79,4,67,16,75,8,20,43,32,...,70,13,62,1,54,29,37,46,58,25
0,img_path,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...
1,timestamps,"[3639.0, 3639.8]","[173.08, 174.08]","[3105.48, 3240.72]","[683.96, 734.36]","[3494.72, 3509.76]","[254.04, 374.08]","[846.68, 865.04]","[1860.88, 1932.44]","[1374.08, 1477.44]",...,"[3300.36, 3309.28]","[539.04, 607.32]","[2670.0, 2761.72]","[0.0, 158.72]","[2327.76, 2387.84]","[1224.24, 1339.64]","[1601.72, 1657.0]","[2000.2, 2062.64]","[2427.88, 2462.24]","[1086.24, 1118.2]"
2,transcription,NaN,NaN,"Well, whether ingested in food or supplemente...",Remember that carbohydrates are a macromolecu...,Here are some questions which process uses tr...,We studied water in our last chapter and we l...,Foods that haven't been stripped of their nut...,Water soluble vitamins are the ones that are ...,Here's some pictures to help you visualize th...,...,Facilitated diffusion is another way that sub...,Many people might choose to rehydrate themsel...,The mitochondria are an organelle that use gl...,"Welcome to Chapter 3, Nutrients, Membrings an...","As we learned in our last chapter, some membr...","Most mammals, including us humans, store fat ...","Unfortunately, this process creates something...",Calcium is a commonly supplemented mineral. Y...,"As you can see in the picture here, these pro...",Complete proteins contain all the essential a...
3,ocr_extracted_text,Transport Across Membranes\n\n+ Exocytosis: a ...,Do you use any of these supplements?\n\n>\n\ng...,Transport Across Membranes\n\n* Membrane trans...,Nutrients: Macronutrients\n\n* Carbohydrates: ...,Which process uses transport proteins to move\...,Nutrients: Macronutrients\n\n» Water\n— Disper...,Nutrients: Macronutrients\n\n* Whole foods\n— ...,Nutrients: Micronutrients\n\n+ Water-soluble v...,NaN,...,Transport Across Membranes\n\n* Passive transp...,Nutrients: Macronutrients\n\n* Sports drinks\n...,Cell Structure\n\n+ Subcellular structures\n\n...,Chapter 3:\n\nNutrients & Membrane Transport\n...,Cell Structure\n\n* Membrane structure\n\n—Flu...,Nutrients: Macronutrients\n\n* Fats: source of...,Nutrients: Macronutrients\n\n+ Fats\n— Hydroge...,Nutrients: Micronutrients\n\n* Calcium\n— Comm...,Cell Structure\n\n* Membrane structure\n— Embe...,Nutrients: Macronutrients\n\n+ Proteins: parts...
4,llava_result,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The image you've provided appears to be a sli...,...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appe

In [7]:
# get llm_long_summary 
llm_long_summary = df_data_extensive.iloc[6].to_dict().values()

# convert to list
llm_long_summary = list(llm_long_summary)
llm_long_summary = llm_long_summary[1:]
logger.info(llm_long_summary)

2024-07-10 12:13:00.507 | INFO     | __main__:<module>:7 - ['Transport Across Membranes: The slide explains the three main types of transport across cell membranes: simple diffusion, facilitated diffusion, and active transport. Simple diffusion involves the movement of substances from an area of higher concentration to an area of lower concentration without energy input. Facilitated diffusion uses carriers to move substances against their concentration gradient, while active transport requires energy provided by ATP to move substances against their concentration gradient. The slide also notes exocytosis, a type of cellular secretion where substances are released from the cell membrane into the extracellular space through the fusion of vesicles with the membrane and expulsion of large molecules outside the cell.', 'The slide appears to be discussing common dietary supplements and their potential benefits as well as side effects. The lecture likely intends to engage students or audience 

In [20]:
# get ocr_only
# Since ocr_only embeddings can't be mapped to floats due to e-02, e-01, we're generating new embeddings 
ocr_only = df_data_extensive.iloc[3].to_dict().values()

# convert to list
ocr_only = list(ocr_only)

ocr_only = ocr_only[1:]

for i in ocr_only:
    logger.info(i)

2024-07-11 15:59:48.353 | INFO     | __main__:<module>:11 - Transport Across Membranes

+ Exocytosis: a membrane-bound vesicle fuses with the
membrane and expels the large molecule outside the cell

(2) Exocytosis


2024-07-11 15:59:48.356 | INFO     | __main__:<module>:11 - Do you use any of these supplements?

>

gingko
kava
ginseng

melatonin

mooo

coconut water

‘$0 YOU WAVE VITAMINS, AND THEN YOU HAVE "HERBAL
SUPPLEMENTS." WHICH ARE WOT FDA APPROVED.

L
‘AND JUST FOR REFERENCE, THE TWINKIE IS.


2024-07-11 15:59:48.356 | INFO     | __main__:<module>:11 - Transport Across Membranes

* Membrane transport
— Plasma membrane:
phospholipid bilayer is
differentially permeable
= Hydrophobic substances


2024-07-11 15:59:48.362 | INFO     | __main__:<module>:11 - Nutrients: Macronutrients

* Carbohydrates: main energy

source Fructose e
+ Possible structural composition: =
— Single-unit simple sugars
(monosaccharides)
~ Many sugar subunits o
(polysaccharides) (s)crapefined 2) Grapes
uy

*

In [21]:
# get ocr_lava ----- NOT EXISTING
lava_only = df_data_extensive.iloc[4].to_dict().values()

lava_only = list(lava_only)

lava_only = lava_only[1:]

concat_result = []

# list to a list of strings
ocr_only = [str(i) for i in ocr_only]

lava_only = [str(i) for i in lava_only]

for i in range(len(ocr_only)):
    concat_result.append(ocr_only[i] + lava_only[i])

# get the embedder model
embedder_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

concat_result_embeddings = text_to_embedding_transformer(concat_result, embedder_model)

In [22]:
# get ocr_transcriptions
# TODO ---- OCR Transcriptions are nans
# ocr_transcriptions = df_data_extensive.iloc[14].to_dict().values()
# # convert to list
# ocr_transcriptions = list(ocr_transcriptions)
# 
# ocr_transcriptions = ocr_transcriptions[1:]
# 
# logger.info(ocr_transcriptions)

In [23]:
# get img_path
img_path = df_data_extensive.iloc[0].to_dict().values()

# convert to list
img_path = list(img_path)
img_path = img_path[1:]

In [24]:
# Embedd with LLM long summary
logger.info(f"Embedd with LLM long summary")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(llm_long_summary)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_extensive_summary = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_extensive_summary.to_csv('df_standard_llm_long_summary.csv', index=False)


2024-07-11 15:59:59.980 | INFO     | __main__:<module>:2 - Embedd with LLM long summary
2024-07-11 16:00:06.564 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[ 0.0569, -0.0283, -0.0086,  ...,  0.3632,  0.3583,  0.0850],
        [-0.2071, -0.1645, -0.0042,  ...,  0.1312, -0.0608,  0.2009],
        [-0.0083,  0.1942, -0.0935,  ...,  0.1645,  0.2800, -0.1498],
        ...,
        [-0.1003, -0.2544,  0.4474,  ..., -0.2105,  0.2100, -0.1704],
        [-0.1289, -0.0912, -0.1340,  ...,  0.0765,  0.2274, -0.0715],
        [-0.2274, -0.1598,  0.0243,  ..., -0.0246,  0.0617, -0.0040]])
2024-07-11 16:00:06.579 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-11 16:00:06.626 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.5148, 0.5050, 0.3735]),
indices=tensor([62, 63,  1])) - GT is keyframe number 3
2024-07-11 16:00:06.626 | INFO     | src.text_embedder.embe

<Figure size 800x2000 with 0 Axes>

In [26]:
import numpy as np
import re
# Embedd with OCR ONLY 
logger.info(f"Embedd with OCR ONLY ")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path


# need to be embedded

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_only)

embedding_model.text_embeddings = embeddings

# already embedded 
#
# if isinstance(embedding_model.text_embeddings, list):
#     for i, text_embedding in enumerate(embedding_model.text_embeddings):
#         logger.info(text_embedding)
#         # Regular expression pattern to find numeric values
#         numeric_pattern = r"[-+]?\d*\.\d+(?:[eE][-+]?\d+)?"
#         
#         # Extract numeric values from the tensor string
#         tensor_values_str = re.findall(numeric_pattern, text_embedding)
#         
#         # Convert the extracted string values to float
#         tensor_values = [float(value) for value in tensor_values_str]
#         
#         # Create a PyTorch tensor from the list of floats
#         embedding_model.text_embeddings[i] = torch.tensor(tensor_values)
#         
# 
# # create one single torch for sim search 
# embedding_model.text_embeddings = torch.stack(embedding_model.text_embeddings, dim=0)
# 
# logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_only = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_only.to_csv('df_standard_ocr_only.csv', index=False)
logger.info('Saved old_df_standard_ocr_only.csv')

2024-07-11 16:00:24.035 | INFO     | __main__:<module>:4 - Embedd with OCR ONLY 
2024-07-11 16:00:29.662 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-11 16:00:29.694 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.3216, 0.2852, 0.2718]),
indices=tensor([ 8, 63, 62])) - GT is keyframe number 3
2024-07-11 16:00:29.694 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:159 - Length of img paths: 85
2024-07-11 16:00:29.694 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - #####GT is keyframe number 3#####
2024-07-11 16:00:29.709 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Max similarity for index 8 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-032-01.jpg
2024-07-11 16:00:29.709 | INFO     | src.text_

<Figure size 800x2000 with 0 Axes>

In [19]:
# Embedd with OCR TRANSCRIPTIONS
# TODO: Doesn't work since transcriptions are nans
logger.info(f"Embedd with TRANSCRIPTIONS")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_transcriptions)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_transcriptions = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_transcriptions.to_csv('df_standard_df_ocr_transcriptions.csv', index=False)

2024-07-08 11:38:54.159 | INFO     | __main__:<module>:2 - Embedd with TRANSCRIPTIONS
2024-07-08 11:39:02.431 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.1037, -0.2244, -0.1416,  ...,  0.0988,  0.2687, -0.2368],
        [-0.1382, -0.2408,  0.0148,  ..., -0.0026, -0.0469, -0.0778],
        [-0.1101, -0.2290,  0.0206,  ...,  0.0204,  0.0192, -0.1317],
        ...,
        [-0.1611, -0.2689,  0.0125,  ...,  0.0216, -0.0170, -0.0902],
        [-0.0916, -0.2179,  0.1103,  ...,  0.0467, -0.0785, -0.1586],
        [-0.0801, -0.1974,  0.0997,  ...,  0.0742, -0.0463, -0.1434]])
2024-07-08 11:39:02.432 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-08 11:39:02.457 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-08 11:39:02.458 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-08 11:39:02

<Figure size 800x2000 with 0 Axes>

In [13]:
# Embedd with OCR LAVA
logger.info(f"Embedd with OCR LAVA")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embedding_model.text_embeddings = concat_result_embeddings

logger.info(embedding_model.text_embeddings)

rows = []

rows = get_results(df)

df_ocr_lava = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_lava.to_csv('df_standard_ocr_lava.csv', index=False)

2024-07-10 12:13:37.901 | INFO     | __main__:<module>:2 - Embedd with OCR LAVA
2024-07-10 12:13:40.168 | INFO     | __main__:<module>:16 - tensor([[ 0.0040,  0.0505, -0.0259,  ...,  0.2132,  0.3487, -0.0278],
        [-0.1964, -0.1597,  0.1473,  ..., -0.0179, -0.0157,  0.1054],
        [ 0.0255,  0.2198, -0.0245,  ...,  0.1483,  0.2668, -0.0646],
        ...,
        [-0.2354, -0.2279,  0.2687,  ..., -0.2134,  0.2025, -0.1898],
        [-0.1530, -0.0735, -0.1421,  ...,  0.1226,  0.3679, -0.1980],
        [-0.0678, -0.1141,  0.0984,  ..., -0.0348,  0.1379, -0.0338]])
2024-07-10 12:13:40.170 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-10 12:13:40.201 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:175 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.3170, 0.2707, 0.2687]),
indices=tensor([63, 62, 16])) - GT is keyframe number 3
2024-07-10 12:13:40.201 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:177 - Le

<Figure size 800x2000 with 0 Axes>